# Plot eigenvalues of Hermitian pencils

The script below plots eigenvalues of a Hermition pencil $A(t) = D + t\mathbf u\mathbf u^\top$.

Input can be specified in one of three ways:
1.  By specifying an input .npz file containing keys `d` and `u`, where `d` contains a 1d array of the diagonal entries.
2.  By specifying $D$ and $\mathbf u$ directly.
3.  By specifying parameters for randomized input.

The order of precedence of these input methods goes from 1 to 3; e.g., if the user specifies both and input file and $D$ and $\mathbf u$, then $D$ and $\mathbf u$ will be loaded from the input file.

In [ ]:
from rk1pert import *
import matplotlib.pyplot as plt
%matplotlib inline

inputfile = None # This supercedes other parameters

D = None
u = None

n = 5
dmin = 0
dmax = 20
tmin = 0
tmax = 20
tstep = .01
plot_asymptotes = True

# If input file is specified, load D and u from there instead
if inputfile is not None:
    inputfile = np.load(inputfile)
    D = np.diag(inputfile['d'])
    u = inputfile['u']
    
if D is None:
    D = rand_diag(n, dmin, dmax)
if u is None:
    u = rand_unit_vector(n)
    
n = len(u)
    
U = np.outer(u, u)
print("u = " + str(u))
print("D = " + str(np.diag(D)))
A = pencil(D, U)

x, y = get_data_points(A, tmin, tmax, tstep)
for i in range(n):
    plt.plot(x, y[i])
plt.xlabel(r"$t$")
plt.ylabel(r"Eigenvalues of $A(t)$")

# Asymptotes
if plot_asymptotes:
    U, s, V = np.linalg.svd(U) # U will now contain an extension of u to an orthonormal basis for R^n
    U = U[:, 1:]
    S = U.T.dot(D).dot(U)
    mu = np.linalg.eigvalsh(S)
    mu.sort()
    mu = mu[::-1]
    print(u"\u03bc = " + str(mu))
    for y in mu:
        plt.plot(x, y * np.ones(len(x)), ':')
    
    plt.plot(x, D.dot(u).dot(u) + x, ':')

plt.text(0, -10, "u = " + str(u) + "\nD = " + str(np.diag(D)) + "\nmu = " + str(mu))
plt.savefig("output.pdf", bbox_inches = "tight") # This makes the text visible in the PDF
plt.show()

np.savez("output.npz", u = u, d = np.diag(D))